# Evaluer tidsepoken for en bok


In [51]:
import dhlab.module_update as mu
import dhlab.text as dh
import pandas as pd
import verk as v
from scipy.spatial.distance import cosine

In [52]:
mu.css()

# Matrix

The matrix of words used to evaluate books

In [172]:
matrix = pd.read_csv("old_vs_new_word_list.csv", index_col = 0)
matrix.sort_values(by=matrix.columns[0], ascending=False).head(20).style.background_gradient(axis = 1)

,1800,1850,1920,1990
af,1.152674,1.098219,0.098675,0.020981
paa,0.650003,0.700560,0.279807,0.009777
saa,0.361886,0.344772,0.142741,0.004964
have,0.221820,0.145100,0.016892,0.009648
havde,0.193529,0.256378,0.025995,0.002352
ere,0.192937,0.091355,0.006076,0.001335
mig,0.181975,0.222869,0.159469,0.007483
efter,0.171206,0.187263,0.174598,0.009219
end,0.166197,0.134345,0.040118,0.003476
deres,0.154052,0.088219,0.043179,0.037609


In [123]:
def eval_books(corpus):
    cts = dh.Counts(corpus)
    for x in cts.counts:
        cts.counts[x] = cts.counts[x]/cts.counts[x].sum()
    return cts
    

In [189]:
v.urns_for_works('e9764ff2f8b1631cbc170f266eb4120cdbcc8f60f6df060f1f0000da19671aac')

['URN:NBN:no-nb_digibok_2012070905001',
 'URN:NBN:no-nb_digibok_2011031012001',
 'URN:NBN:no-nb_digibok_2012070408109',
 'URN:NBN:no-nb_digibok_2014032738014',
 'URN:NBN:no-nb_digibok_2006082800014',
 'URN:NBN:no-nb_digibok_2011120108006',
 'URN:NBN:no-nb_digibok_2006082400043',
 'URN:NBN:no-nb_digibok_2008061900071',
 'URN:NBN:no-nb_digibok_2010070106207']

In [ ]:
'e9764ff2f8b1631cbc170f266eb4120cdbcc8f60f6df060f1f0000da19671aac': {'URN:NBN:no-nb_digibok_2012070408109': ('1800',
   nan),
  'URN:NBN:no-nb_digibok_2012070905001': ('1800', nan),
  'URN:NBN:no-nb_digibok_2010070106207': ('1800', nan),
  'URN:NBN:no-nb_digibok_2006082400043': ('1800', nan),
  'URN:NBN:no-nb_digibok_2006082800014': ('1800', nan),
  'URN:NBN:no-nb_digibok_2011031012001': ('1800', nan),
  'URN:NBN:no-nb_digibok_2011120108006': ('1800', nan),
  'URN:NBN:no-nb_digibok_2008061900071': ('1800', nan),
  'URN:NBN:no-nb_digibok_2014032738014': ('1800', nan)}

In [382]:
corpus = dh.CorpusFromIdentifiers(identifiers = v.urns_for_works('1d4ea7e02197c411eb6670d943664e88beded896e11093af4063c9a5c13d7211') )

In [383]:
eb = eval_books(corpus)

In [384]:
check = eb.counts.loc[[x for x in eb.counts.index if x.isalpha()]]

In [385]:
check

,URN:NBN:no-nb_digibok_2010070106207,URN:NBN:no-nb_digibok_2006082400043,URN:NBN:no-nb_digibok_2008060904035,URN:NBN:no-nb_digibok_2010032213006,URN:NBN:no-nb_digibok_2009022600051,URN:NBN:no-nb_digibok_2011031012001,URN:NBN:no-nb_digibok_2008051504087,URN:NBN:no-nb_digibok_2012070905001,URN:NBN:no-nb_digibok_2012070408109,URN:NBN:no-nb_digibok_2008061900071
og,0.031948,0.025484,0.025875,0.029314,0.025954,0.028205,0.031609,0.031933,0.031443,0.032117
han,0.019946,0.010403,0.005025,0.016668,0.005034,0.025641,0.019443,0.019920,0.019561,0.020049
det,0.018953,0.013357,0.017713,0.018320,0.017600,0.012821,0.019098,0.018973,0.019247,0.019089
i,0.013839,0.018837,0.023630,0.015428,0.023848,0.017949,0.013259,0.013733,0.012681,0.013791
var,0.013058,0.014588,0.006130,0.012582,0.006105,0.005128,0.013283,0.013053,0.012817,0.013130
...,...,...,...,...,...,...,...,...,...,...
Fortællinger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000016
NASJONALLITTERATUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000016
Sige,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000016
tU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000016


In [386]:
authors = ["bjørnstjerne bjørnson", "henrik ibsen", "amalie skram", "alexander kielland", "jonas lie", "camilla collett"]

In [387]:

def evaluate_corpus_norwegian(corpus=None, matrix=None, top_number=50):
    """corpus is a dh.Corpus, matrix an evaluation matrix with columns corresponding to years, and rows a word weighted for each year"""
    
    # normalize corpus
    check = dh.Counts(corpus)
    
    for x in check.counts:
        check.counts[x] = check.counts[x]/check.counts[x].sum()
        
    # values to be returned
    vals = dict()
    
    # loop through the corpus and evaluate each book against the matrix
    check_df = check.counts
    for x in check_df:
        # sort the book according to frequency and keep the top_number
        targets = check_df.iloc[check_df[x].fillna(0).to_numpy().nonzero()].index
        res = {x:[]}
        for z in matrix:
            
            # check which words are both in the book as well as in the matrix
            
            words = [w for w in matrix[z].sort_values(ascending=False).index if w in targets][:20]
            
            # evaluate matrix against book using scipy cosine
            try:
                df = pd.concat([matrix[z].loc[words], check_df[x].loc[words]], axis = 1)
                df.columns = [0, 1]
                res[x].append((z, cosine(df[0], df[1])))
            except ZeroDivisionError:
                pass
        # sort the result and keep the best
        res[x].sort(key= lambda x: x[1], reverse = True)
        vals[x] = res[x][0]
        
    return vals

In [388]:
c = dh.Counts(corpus)

In [389]:
c.counts

,URN:NBN:no-nb_digibok_2010070106207,URN:NBN:no-nb_digibok_2006082400043,URN:NBN:no-nb_digibok_2012070905001,URN:NBN:no-nb_digibok_2012070408109,URN:NBN:no-nb_digibok_2009022600051,URN:NBN:no-nb_digibok_2010032213006,URN:NBN:no-nb_digibok_2008061900071,URN:NBN:no-nb_digibok_2011031012001,URN:NBN:no-nb_digibok_2008051504087,URN:NBN:no-nb_digibok_2008060904035
",",8336.0,5578.0,8334.0,8463.0,2335.0,10850.0,8299.0,61.0,5368.0,2326.0
.,5019.0,3537.0,5037.0,4588.0,1022.0,5684.0,5010.0,66.0,3518.0,1132.0
og,4086.0,2381.0,4083.0,3702.0,727.0,4541.0,4080.0,33.0,2658.0,726.0
»,2587.0,717.0,2584.0,2442.0,237.0,2630.0,2577.0,44.0,1759.0,239.0
«,2582.0,797.0,2582.0,2442.0,237.0,2638.0,2576.0,45.0,1759.0,239.0
...,...,...,...,...,...,...,...,...,...,...
ægg-rike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
ændeløs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
jantasus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
maisang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [394]:
import importlib
importlib.reload(ev)

<module 'evaluate_corpus' from '/mnt/disk1/Github/Historisk_ordbok/verksregister/evaluate_corpus.py'>

In [396]:
ev.evaluate_corpus_norwegian(corpus = corpus, matrix = matrix, top_number=100)

{'URN:NBN:no-nb_digibok_2011031012001': ('1990', 0.33350212401357215),
 'URN:NBN:no-nb_digibok_2008051504087': ('1800', 0.7400790365646015),
 'URN:NBN:no-nb_digibok_2010032213006': ('1800', 0.7259699047100436),
 'URN:NBN:no-nb_digibok_2009022600051': ('1800', 0.330405421079798),
 'URN:NBN:no-nb_digibok_2008060904035': ('1800', 0.32213967578803293),
 'URN:NBN:no-nb_digibok_2010070106207': ('1800', 0.7116082336925302),
 'URN:NBN:no-nb_digibok_2006082400043': ('1850', 0.34351699293136906),
 'URN:NBN:no-nb_digibok_2008061900071': ('1850', 0.7313992917800565),
 'URN:NBN:no-nb_digibok_2012070905001': ('1800', 0.707800188703967),
 'URN:NBN:no-nb_digibok_2012070408109': ('1990', 0.3448031940520595)}

In [380]:
evaluate_corpus_norwegian(corpus= corpus, matrix = matrix, top_number=10)

{'URN:NBN:no-nb_digibok_2014032738014': ('1850', 0.3700948900715253),
 'URN:NBN:no-nb_digibok_2012070905001': ('1800', 0.707800188703967),
 'URN:NBN:no-nb_digibok_2012070408109': ('1990', 0.3448031940520595),
 'URN:NBN:no-nb_digibok_2006082400043': ('1850', 0.34351699293136906),
 'URN:NBN:no-nb_digibok_2006082800014': ('1850', 0.7542104712618121),
 'URN:NBN:no-nb_digibok_2010070106207': ('1800', 0.7116082336925302),
 'URN:NBN:no-nb_digibok_2008061900071': ('1850', 0.7313992917800565),
 'URN:NBN:no-nb_digibok_2011031012001': ('1990', 0.33350212401357215),
 'URN:NBN:no-nb_digibok_2011120108006': ('1850', 0.5884523346199722)}

In [381]:
evaluate_corpus_norwegian(corpus= corpus, matrix = matrix, top_number=200)

{'URN:NBN:no-nb_digibok_2008061900071': ('1850', 0.7313992917800565),
 'URN:NBN:no-nb_digibok_2012070905001': ('1800', 0.707800188703967),
 'URN:NBN:no-nb_digibok_2012070408109': ('1990', 0.3448031940520595),
 'URN:NBN:no-nb_digibok_2014032738014': ('1850', 0.3700948900715253),
 'URN:NBN:no-nb_digibok_2006082800014': ('1850', 0.7542104712618121),
 'URN:NBN:no-nb_digibok_2006082400043': ('1850', 0.34351699293136906),
 'URN:NBN:no-nb_digibok_2010070106207': ('1800', 0.7116082336925302),
 'URN:NBN:no-nb_digibok_2011120108006': ('1850', 0.5884523346199722),
 'URN:NBN:no-nb_digibok_2011031012001': ('1990', 0.33350212401357215)}

In [196]:
corpus.corpus[["urn", 'year']]

,urn,year
0,URN:NBN:no-nb_digibok_2012070905001,1963
1,URN:NBN:no-nb_digibok_2011031012001,1950
2,URN:NBN:no-nb_digibok_2012070408109,1941
3,URN:NBN:no-nb_digibok_2014032738014,1971
4,URN:NBN:no-nb_digibok_2006082800014,1969
5,URN:NBN:no-nb_digibok_2011120108006,1955
6,URN:NBN:no-nb_digibok_2006082400043,1969
7,URN:NBN:no-nb_digibok_2008061900071,1996
8,URN:NBN:no-nb_digibok_2010070106207,1957


In [245]:
import evaluate_corpus as ev

In [260]:
import importlib
importlib.reload(ev)

<module 'evaluate_corpus' from '/mnt/disk1/Github/Historisk_ordbok/verksregister/evaluate_corpus.py'>

In [361]:
c = dh.CorpusFromIdentifiers(v.urns_for_works('e9764ff2f8b1631cbc170f266eb4120cdbcc8f60f6df060f1f0000da19671aac'))

In [362]:
c.corpus

,dhlabid,title,authors,urn,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype
0,100098258,Bondefortellinger,"Bjørnson , Bjørnstjerne / Sørensen , Henrik",URN:NBN:no-nb_digibok_2012070905001,oai:nb.bibsys.no:990228733684702202,f07f0fa571702b6f1bb80e1044e08eb0,,Oslo,19630101,1963,Gyldendal,nob,,839.93,fiction,Skjønnlitteratur,digibok
1,100042895,Bondefortellinger,"Bjørnson , Bjørnstjerne / Lowzow , Ingrid",URN:NBN:no-nb_digibok_2011031012001,oai:nb.bibsys.no:990309698914702202,6441ffb61ce0fceb77a1097b87389cb2,,Oslo,19500101,1950,Norsk jord,nob,,,,Uklassifisert,digibok
2,100087517,Bondefortællinger,"Bjørnson , Bjørnstjerne / Sørensen , Henrik",URN:NBN:no-nb_digibok_2012070408109,oai:nb.bibsys.no:990803970694702202,c403df21819f3334555a9b172229682f,,Oslo,19410101,1941,Gyldendal,nob,,,fiction,Skjønnlitteratur,digibok
3,100200301,En glad gutt,"Bjørnson , Bjørnstjerne / Haakonsen , Daniel",URN:NBN:no-nb_digibok_2014032738014,oai:nb.bibsys.no:998640398084702202,a594b7d2a8ae51ac6a9b092a2aebc371,8205000409,Oslo,19710101,1971,Gyldendal,nob,bondefortellinger / lærebøker / videregående /...,,,Uklassifisert,digibok
4,100508388,En glad gutt,"Bjørnson , Bjørnstjerne / Sørensen , Henrik",URN:NBN:no-nb_digibok_2006082800014,oai:nb.bibsys.no:999301056054702202,13363841b7ec0b1dbde55614a5c74b6d,,Oslo,19690101,1969,Gyldendal,nob,,839.93,fiction,Skjønnlitteratur,digibok
5,100065000,En glad gutt ; Brudeslåtten ; Blakken ; Trond,"Bjørnson , Bjørnstjerne / Bjørnson , Bjørnstje...",URN:NBN:no-nb_digibok_2011120108006,oai:nb.bibsys.no:999302335304702202,a10cab4af16fa3a1ccb9cdc0fcd1339c,,Oslo,19550101,1955,Gyldendal,nob,skjønnlitteratur / roman,,,Uklassifisert,digibok
6,100508640,Fortellinger,"Bjørnson , Bjørnstjerne / Seip , Didrik Arup",URN:NBN:no-nb_digibok_2006082400043,oai:nb.bibsys.no:999407142644702202,f629cbc08f7ed4341c97756bc7d85938,,Oslo,19690101,1969,Gyldendal,nob,norske / fortellinger / 1860-tallet,839.93,fiction,Skjønnlitteratur,digibok
7,100347525,Bondefortellinger,"Bjørnson , Bjørnstjerne / Gundersen , Dag / Ki...",URN:NBN:no-nb_digibok_2008061900071,oai:nb.bibsys.no:999616277604702202,9381c6d4b3cf72c93b7d4c60d619ef42,8205243263,[Oslo],19960101,1996,Gyldendal,nob,skjønnlitteratur,839.8208,fiction,Skjønnlitteratur,digibok
8,100513560,Bondefortellinger,"Bjørnson , Bjørnstjerne / Sørensen , Henrik",URN:NBN:no-nb_digibok_2010070106207,oai:nb.bibsys.no:999706661284702202,ef9ddd477c188acb0167079325cabe9a,,Oslo,19570101,1957,Gyldendal,nob,,839.93,fiction,Skjønnlitteratur,digibok


In [324]:
dh.Counts??

Init signature: dh.Counts(corpus=None, words=None)
Source:        
class Counts():
    """Provide counts for a corpus - shouldn't be too large"""

    def __init__(self, corpus=None, words=None):
        if corpus is None and words is None:
            self.counts = None
        elif not corpus is None:
            # count - if words is none result will be as if counting all words
            # in the corpus
            self.counts = get_document_frequencies(
                urns=urnlist(corpus), cutoff=0, words=words)
File:           ~/anaconda3/lib/python3.8/site-packages/dhlab/text/conc_coll.py
Type:           type
Subclasses:     


In [363]:
cnts = dh.Counts(c)

In [364]:
cnts.counts

,URN:NBN:no-nb_digibok_2012070408109,URN:NBN:no-nb_digibok_2012070905001,URN:NBN:no-nb_digibok_2008061900071,URN:NBN:no-nb_digibok_2011120108006,URN:NBN:no-nb_digibok_2006082400043,URN:NBN:no-nb_digibok_2006082800014,URN:NBN:no-nb_digibok_2010070106207,URN:NBN:no-nb_digibok_2014032738014,URN:NBN:no-nb_digibok_2011031012001
",",8463.0,8334.0,8299.0,4262.0,5578.0,2462.0,8336.0,2553.0,61.0
.,4588.0,5037.0,5010.0,2449.0,3537.0,1282.0,5019.0,1468.0,66.0
og,3702.0,4083.0,4080.0,2086.0,2381.0,798.0,4086.0,1211.0,33.0
»,2442.0,2584.0,2577.0,888.0,717.0,693.0,2587.0,685.0,44.0
«,2442.0,2582.0,2576.0,896.0,797.0,711.0,2582.0,695.0,45.0
...,...,...,...,...,...,...,...,...,...
Storlia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
bygda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
døra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
sjøl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [267]:
cnts

In [365]:
cnts.counts[cnts.counts.columns[0]]

,          8463.0
.          4588.0
og         3702.0
»          2442.0
«          2442.0
            ...  
Storlia       NaN
bygda         NaN
døra          NaN
sjøl          NaN
sått          NaN
Name: URN:NBN:no-nb_digibok_2012070408109, Length: 25553, dtype: float64

In [325]:
cnts.counts = cnts.counts.fillna(0)

In [366]:
for x in cnts.counts:
    cnts.counts[x] = cnts.counts[x]/cnts.counts[x].sum()

In [367]:
cnts.counts[cnts.counts.columns[3]].index

Index([',', '.', 'og', '»', '«', 'han', 'det', ';', '—', 'så',
       ...
       'tittelblad', 'tofta', 'trykkeri', 'vatnet', 'åra', 'Storlia', 'bygda',
       'døra', 'sjøl', 'sått'],
      dtype='object', length=25553)

In [368]:
col = cnts.counts.columns

In [370]:
cnts.counts.iloc[cnts.counts[col[0]].dropna()]

,URN:NBN:no-nb_digibok_2012070408109,URN:NBN:no-nb_digibok_2012070905001,URN:NBN:no-nb_digibok_2008061900071,URN:NBN:no-nb_digibok_2011120108006,URN:NBN:no-nb_digibok_2006082400043,URN:NBN:no-nb_digibok_2006082800014,URN:NBN:no-nb_digibok_2010070106207,URN:NBN:no-nb_digibok_2014032738014,URN:NBN:no-nb_digibok_2011031012001
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137
...,...,...,...,...,...,...,...,...,...
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137
",",0.071882,0.065179,0.065328,0.066844,0.059701,0.06683,0.065178,0.06835,0.052137


In [373]:
targets = cnts.counts.iloc[cnts.counts[col[1]].fillna(0).to_numpy().nonzero()].index

In [374]:
words = [w for w in matrix["1800"].sort_values(ascending=False).index if w in targets][:20]

In [375]:
matrix["1800"].loc[words]

af       1.152674
paa      0.650003
saa      0.361886
have     0.221820
havde    0.193529
mig      0.181975
efter    0.171206
end      0.166197
deres    0.154052
e        0.149331
nu       0.148931
maa      0.141410
ogsaa    0.136945
os       0.130529
vilde    0.128880
dog      0.124880
disse    0.124192
kunde    0.121903
ei       0.121103
hvad     0.119663
Name: 1800, dtype: float64

In [376]:
cnts.counts[cnts.counts.columns[1]].loc[words]

af       0.000188
paa      0.000219
saa      0.000313
have     0.000109
havde    0.000039
mig      0.000274
efter    0.002894
end      0.000047
deres    0.000235
e        0.000008
nu       0.000274
maa      0.000141
ogsaa    0.000086
os       0.000063
vilde    0.000055
dog      0.000868
disse    0.000360
kunde    0.000031
ei       0.000242
hvad     0.000070
Name: URN:NBN:no-nb_digibok_2012070905001, dtype: float64

In [331]:
pd.concat([pd.DataFrame(matrix["1800"].loc[words]), pd.DataFrame(cnts.counts[cnts.counts[cnts.counts.columns[0]].loc[words]])], axis = 1)

KeyError: "None of [Float64Index([                  0.0,                   0.0,\n                                0.0,                   0.0,\n                                0.0,                   0.0,\n                                0.0,                   0.0,\n                                0.0,                   0.0,\n                                0.0,                   0.0,\n                                0.0,                   0.0,\n                                0.0,                   0.0,\n              0.0008547008547008547,                   0.0,\n                                0.0,                   0.0],\n             dtype='float64')] are in the [columns]"

In [261]:
ev.evaluate_corpus_norwegian(corpus=dh.CorpusFromIdentifiers(v.urns_for_works('e9764ff2f8b1631cbc170f266eb4120cdbcc8f60f6df060f1f0000da19671aac')),
                             matrix=matrix,
                            top_number=30)

ValueError: Length mismatch: Expected axis has 10 elements, new values have 2 elements

In [257]:
ev.evaluate_corpus_norwegian(corpus=dh.CorpusFromIdentifiers(v.urns_for_works('e9764ff2f8b1631cbc170f266eb4120cdbcc8f60f6df060f1f0000da19671aac')),
                             matrix=matrix,
                            top_number=20)

AttributeError: 'Counts' object has no attribute 'index'

In [226]:
matrix.loc['efter']['1800']

0.17120574997847726